In [ ]:
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import math
import random
import lightgbm
from scipy.stats import probplot, pearsonr
from sklearn.model_selection import StratifiedKFold 
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


# Ubiquant Market Prediction

[Соревнование на Kaggle](https://www.kaggle.com/c/ubiquant-market-prediction)


1. Обзор данных
   - Описание датасета 
   - Target
   - Investment ID
   - Time Id
   
    
2. Выбор признаков
   - Корелляция
   - Lightbgm
    
   
3. Обучение модели
   - Linear Regression
   - DNN
   
   
4. Future work


<h1>
    <div style="color:black;
           border-radius:5px;
           background-color:#629b50">
1.Обзор данных
</h1>

## Описание датасета

Я использовала датасет [в формате parquet](https://www.kaggle.com/robikscube/ubiquant-parquet) для более быстрой загрузки данных.

**train.parquet**:

* `row_id` - A unique identifier for the row.

* `time_id` - The ID code for the time the data was gathered. The time IDs are in order, but the real time between the time IDs is not constant and will likely be shorter for the final private test set than in the training set.

* `investment_id`- The ID code for an investment. Not all investment have data in all time IDs.

* `target` - The target.

* `[f_0:f_299]` - Anonymized features generated from market data.


In [ ]:
colors = sns.color_palette('gist_earth_r', 20)

In [ ]:
colors

In [ ]:
train = pd.read_parquet('../input/ubiquant-parquet/train.parquet', engine='pyarrow') 
train.head()

In [ ]:
train.info()

In [ ]:
print('Rows and Columns in train dataset:', train.shape)
print('Missing values in train dataset:', sum(train.isnull().sum()))

### Target

In [ ]:
def dist_target(target, title, label):

    fig, axes = plt.subplots(ncols=2, figsize=(24, 6), dpi=100)
    sns.distplot(x=target, bins=50, color= '#629b50', ax=axes[0])
    axes[0].axvline(x=np.mean(target), color='black')
    axes[0].set_title(title)
    axes[0].set_xlabel(label)
    axes[0].set_ylabel('Частота')
    
    probplot(target, plot=axes[1])
    axes[1].get_lines()[0].set_marker('p')
    axes[1].get_lines()[0].set_color('#629b50')
    axes[1].get_lines()[1].set_color('black')

plt.show()


In [ ]:
dist_target(train['target'], title='Распределение значений target', label='target')

In [ ]:
train['target'].describe(percentiles=[0.01, 0.25, 0.5, 0.75, 0.99]).to_frame().T

Распределение имеет форму, похожую на нормальное, но с более тяжелыми хвостами.

### Investment ID

In [ ]:
plt.figure(figsize = (12, 5))
sns.histplot(x=train.groupby(['investment_id'])['target'].count(), bins=50, color='#629b50')
plt.title('Распределение количества записей для Investment ID')
plt.xlabel('Количество записей')
plt.ylabel('Частота')
plt.show()

Выше я смотрела на распределение значений таргета. Посмотрим теперь, как будет выглядеть распределение значений таргета, усредненных по активам:

In [ ]:
mean_target = train.groupby(['investment_id'])['target'].mean()
dist_target(mean_target, title='Распределение средних значений target по Investment ID', label='Среднее значение')

In [ ]:
mean_target.describe([0.01, 0.25, 0.5, 0.75, 0.99]).to_frame().T

По сравнению с распределением всех значений таргета, в этом случае уменьшается дисперсия. 

Посмотрим, как среднее значение и STD зависят от количества наблюдений:

In [ ]:
axes = sns.jointplot(
              x=train.groupby(['investment_id'])['target'].count().values, 
              y=train.groupby(['investment_id'])['target'].mean(), 
              kind='reg', 
              height=8, 
              joint_kws={'line_kws':{'color':'black'}},
              color='#629b50'
              )

axes.ax_joint.set_xlabel('Количество наблюдений')
axes.ax_joint.set_ylabel('Среднее значение target')

plt.title('Зависимость среднего значения target от количества наблюдений', y=1.2)
plt.show()

In [ ]:
axes = sns.jointplot(
              x=train.groupby(['investment_id'])['target'].count().values, 
              y=train.groupby(['investment_id'])['target'].std(), 
              kind='reg', 
              height=8, 
              joint_kws={'line_kws':{'color':'black'}},
              color='#629b50'
              )

axes.ax_joint.set_xlabel('Количество наблюдений')
axes.ax_joint.set_ylabel('STD target')

plt.title('Зависимость STD target от количества наблюдений', y=1.2)
plt.show()

Чем больше наблюдений, тем меньше разброс средних значений.

In [ ]:
fig, axes = plt.subplots(1, 5, figsize=(10, 2.5), dpi=100, sharex=True, sharey=True)

for i, (ax, investment_id) in enumerate(zip(axes.flatten(), np.random.choice(train["investment_id"].unique(),5))):
    x = train.loc[train.investment_id==investment_id, 'target']
    ax.hist(x, alpha=0.7, bins=55, density=True, stacked=True, label=str(investment_id), 
            color = random.choice(sns.color_palette('gist_earth_r', 20)))
    ax.set_title(investment_id)
    
plt.suptitle('Распределение значений target для случайных Investment ID', y=1.1)  
plt.show()

### Time ID

In [ ]:
fig, axes = plt.subplots(3, figsize=(20, 10), sharex=True)

train.groupby('time_id')['investment_id'].nunique().plot(color=random.choice(colors), ax=axes[0])
axes[0].set_title('Количество уникальных активов по времени')

train.groupby('time_id')['target'].mean().plot(color=random.choice(colors), ax=axes[1])
axes[1].set_title("Среднее значение target по времени")
axes[1].axhline(y=np.mean(mean_target), color='black', linestyle='--', label="mean")


train.groupby('time_id')['target'].std().plot(color=random.choice(colors), ax=axes[2])
axes[2].set_title("STD target по времени")
axes[2].axhline(y=np.mean(train.groupby('time_id')['target'].std()), color='black', linestyle='--', label="mean")

plt.show()

In [ ]:
train[['investment_id', 'time_id']] \
    .loc[train['investment_id'] < 100] \
    .plot.scatter('time_id', 'investment_id', figsize=(20, 6), s=0.5, color='grey')

plt.title('Плотность записей по времени для Investment ID')
plt.show()

Выводы:
* В момент времени около 400 есть аномалии, для многих активов отсутствуют записи.
* По этим графикам видно, что существует некоторая обратная зависимость между средним значением таргета и количеством уникальных записей активов в этот день. Возможно, имеет смысл добавить к каждому time_id количество уникальных активов в этот момент. 
* Временной ряд средних значений таргета не имеет тренда
* Сезонности на первый взгляд нет, однако промежутки между time_id не постоянны и нельзя сказать точно.

<h1>
    <div style="color:black;
           border-radius:5px;
           background-color:#629b50">
2. Выбор признаков
</h1>

Я использовала 1% рандомно выбранных данных, чтобы ускорить процесс.  
Выбор признаков осложняется тем, что они анонимны.

In [ ]:
features = train.columns[4:]

In [ ]:
features_t = train.columns[3:]

### Корелляция признаков между собой

In [ ]:
sample_corr = train.sample(frac=0.01, random_state=42)
correlation = sample_corr[features_t].corr()

sns.clustermap(correlation, figsize=(20, 20), cmap='Greens')

Видно, что есть сильно скореллированные между собой признаки, как отрицательно, так и положительно. Возможно, от таких пар стоит избавиться - они передают одинаковую информацию.

### Корелляция признаков и таргета

In [ ]:
plt.figure(figsize=(12, 5))
sns.histplot(x=correlation['target'].iloc[1:], bins=50, color='#629b50')
plt.title('Распределение значений корелляции фичей и таргета')
plt.xlabel('Корелляция')
plt.ylabel('Частота')
plt.show()

In [ ]:
correlation['target'].iloc[1:].describe([0.01, 0.25, 0.5, 0.75, 0.99]).to_frame().T

Большой корелляции между фичами и таргетом не наблюдается. Посмотрим на топ 20 фичей по корелляции, но выбрать лучшие признаки это не поможет.

In [ ]:
feat_importances_t = correlation['target'].iloc[1:]
feat_importances_t.nlargest(20).plot(kind='barh', figsize=(12, 6),
                                   color=sns.color_palette('Greens_r', 20)).invert_yaxis()
plt.title('Топ 20 признаков')
plt.show()

Уже выяснили, что есть связь между количеством активов в time_id и средним значением таргета.  
Возможно есть зависимость количества наблюдений для актива и фичами. Заменим investment ID на количество наблюдений

In [ ]:
s_train=train.sample(frac=0.01, random_state = 42)

In [ ]:
obs_by_asset = s_train.groupby(['investment_id'])['target'].count().to_dict()
target = s_train.investment_id.copy().replace(obs_by_asset).astype(np.int16)
features = s_train.columns[4:]

del(obs_by_asset)

In [ ]:
corrs = []
for col in features:
    corr = np.corrcoef(target, s_train[col])[0][1]
    corrs.append(corr)
    
del(target)

In [ ]:
feat_importances = pd.Series(corrs, index=features)
feat_importances.nlargest(20).plot(kind='barh', figsize=(12, 6),
                                   color=sns.color_palette('Greens_r', 20)).invert_yaxis()
plt.title('Top 20 признаков')
plt.show()

Действительно есть корелляция для нескольких признаков.  
Можно попробовать добавить еще признаки - количество наблюдений актива за все время.

## Lgbm Feature Importance 


Еще один способ найти наиболее значимые признаки - обучить модель и взять признаки с наибольшим весом.

In [ ]:

features = [f'f_{i}' for i in range(300)]
    
target = 'target'

In [ ]:
seed = 8
folds = 5

skf = StratifiedKFold(folds, shuffle = True, random_state = seed)

for train_index, test_index in skf.split(s_train, s_train['investment_id']):
    train1 = s_train.iloc[train_index]
    valid1 = s_train.iloc[test_index]
    
    lgbm = LGBMRegressor(
        num_leaves=2 ** np.random.randint(3, 8),
        learning_rate = 10 ** (-np.random.uniform(0.1, 2)),
        n_estimators = 1000,
        min_child_samples = 1000, 
        subsample=np.random.uniform(0.5, 1.0), 
        subsample_freq=1,
        n_jobs= -1
    )

    lgbm.fit(train1[features], train1[target], eval_set = (valid1[features], valid1[target]), early_stopping_rounds = 10)

In [ ]:
feature_imp = pd.DataFrame(sorted(zip(lgbm.feature_importances_, s_train.columns)), columns=['Value', 'Feature']).nlargest(20, 'Value')

plt.figure(figsize=(12, 6))
sns.barplot(x='Value', y='Feature', data=feature_imp.sort_values(by='Value', ascending=False),  palette='Greens_r')
plt.title('LightGBM Important Features')
plt.tight_layout()
plt.show()

Посмотрим на распределение топ-9 фичей

In [ ]:
top_f = feature_imp.iloc[:9].Feature.to_list()

In [ ]:
cols = ["row_id","time_id","investment_id","target"].append(top_f)

In [ ]:
df = pd.DataFrame(s_train, columns= cols)
df.head()

In [ ]:
fig, ax = plt.subplots(3,3, figsize=(15, 15))
for i, sample in enumerate(top_f):
    sns.distplot(df[sample], ax=ax[math.floor(i/3),i%3], color=random.choice(sns.color_palette('gist_earth_r', 20))).set_title(f'{sample} Distribution')
fig.suptitle('Top 9 Feature Density Plot', y=1, size=16) 
fig.tight_layout()
fig.show()

Фичи имеют разное распределение.

<h1>
    <div style="color:black;
           border-radius:5px;
           background-color:#629b50">
3. Обучение модели
</h1>

### Линейная регрессия

Работает очень плохо(

In [ ]:
Y= s_train['target']
X = s_train[s_train.columns[4:]]

In [ ]:
model = LinearRegression().fit(X, Y)
model.score(X,Y)

In [ ]:
Y_pred = model.predict(X)
lin = pd.DataFrame({'Actual': Y, 'Predicted': Y_pred.flatten()})
display(lin)

### DNN

На основе этой модели были созданы почти все модели с хорошим скором на этом соревновании, поэтому я взяла ее. Архитектура не моя, но она показалась мне логичной. 

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
from scipy import stats
from tensorflow.python.ops import math_ops
from tensorflow.python.keras import backend as K

Создадим IntegerLookup layer для investment_id (количество уникальных значений < максимальный индекс). В файле investment_ids.csv содержатся уникальные значения investment_id. 

In [ ]:
investment_ids = pd.read_csv('../input/ump-combinatorialpurgedgroupkfold-tf-record/investment_ids.csv')
investment_id_size = len(investment_ids) + 1

with tf.device('cpu'):
    investment_id_lookup_layer = layers.IntegerLookup(max_tokens=investment_id_size)
    investment_id_lookup_layer.adapt(investment_ids)

### Tensorflow dataset

In [ ]:
def decode_function(record_bytes):
    return tf.io.parse_single_example(
      record_bytes,
      {
          'features': tf.io.FixedLenFeature([300], dtype=tf.float32),
          'time_id': tf.io.FixedLenFeature([], dtype=tf.int64),
          'investment_id': tf.io.FixedLenFeature([], dtype=tf.int64),
          'target': tf.io.FixedLenFeature([], dtype=tf.float32)
      }
  )

def preprocess(item):
    return (item['investment_id'], item['features']), item['target']

def make_dataset(file_paths, batch_size=4096, mode='train'):
    ds = tf.data.TFRecordDataset(file_paths)
    ds = ds.map(decode_function)
    ds = ds.map(preprocess)
    if mode == 'train':
        ds = ds.shuffle(batch_size * 4)
    ds = ds.batch(batch_size).cache().prefetch(tf.data.AUTOTUNE)
    return ds

### Модель

Категориальные (investment_id) и численные (f_i) признаки подаются на вход отдельно. 

Категориальный признак довольно разреженный (есть много investment_id), в этом случае будет хорошо применить эмбеддинг. В остальном сеть состоит из слоев с активацией 'swish' (есть мнение, что она лучше чем ReLu) и дропаута (чтобы избежать переобучения).

In [ ]:
def correlation(x, y, axis=-2):
    """Metric returning the Pearson correlation coefficient of two tensors over some axis, default -2."""
    x = tf.convert_to_tensor(x)
    y = math_ops.cast(y, x.dtype)
    n = tf.cast(tf.shape(x)[axis], x.dtype)
    xsum = tf.reduce_sum(x, axis=axis)
    ysum = tf.reduce_sum(y, axis=axis)
    xmean = xsum / n
    ymean = ysum / n
    xvar = tf.reduce_sum(tf.math.squared_difference(x, xmean), axis=axis)
    yvar = tf.reduce_sum(tf.math.squared_difference(y, ymean), axis=axis)
    cov = tf.reduce_sum((x - xmean) * (y - ymean), axis=axis)
    corr = cov / tf.sqrt(xvar * yvar)
    
    return tf.constant(1.0, dtype=x.dtype) - corr


def get_model():
    investment_id_inputs = tf.keras.Input((1, ), dtype=tf.uint16)
    features_inputs = tf.keras.Input((300, ), dtype=tf.float16)
    
    investment_id_x = investment_id_lookup_layer(investment_id_inputs)
    investment_id_x = layers.Embedding(investment_id_size, 32, input_length=1)(investment_id_x)
    investment_id_x = layers.Reshape((-1, ))(investment_id_x)
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
    investment_id_x = layers.Dropout(0.1)(investment_id_x)
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
    investment_id_x = layers.Dropout(0.1)(investment_id_x)
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
    investment_id_x = layers.Dropout(0.1)(investment_id_x)
    
    feature_x = layers.Dense(256, activation='swish')(features_inputs)
    feature_x = layers.Dropout(0.1)(feature_x)
    feature_x = layers.Dense(256, activation='swish')(feature_x)
    feature_x = layers.Dropout(0.1)(feature_x)
    feature_x = layers.Dense(256, activation='swish')(feature_x)
    feature_x = layers.Dropout(0.1)(feature_x)
    
    x = layers.Concatenate(axis=1)([investment_id_x, feature_x])
    x = layers.Dense(512, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dropout(0.1)(x)
    x = layers.Dense(128, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dropout(0.1)(x)
    x = layers.Dense(32, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dropout(0.1)(x)
    
    output = layers.Dense(1)(x)
    
    rmse = keras.metrics.RootMeanSquaredError(name="rmse")
    model = tf.keras.Model(inputs=[investment_id_inputs, features_inputs], outputs=[output])
    model.compile(optimizer=tf.optimizers.Adam(0.001), loss='mse', metrics=['mse', "mae", "mape", rmse, correlation])
    
    return model

Архитектура 

In [ ]:
model = get_model()
model.summary()
keras.utils.plot_model(model, show_shapes=True)

Чтобы ускорить процесс, я взяла заранее разделенные на фолды данные, которые [любезно предоставил](https://www.kaggle.com/lonnieqin/ump-combinatorialpurgedgroupkfold-tf-record) автор данного решения. Они разделены по принципу CombinatorialPurgedGroupKFold.

In [ ]:
 tf_record_dataset_path = '../input/ump-combinatorialpurgedgroupkfold-tf-record/'

In [ ]:
%%time
models = []
for i in range(5):
    train_path = f'{tf_record_dataset_path}fold_{i}_train.tfrecords'
    valid_path = f'{tf_record_dataset_path}fold_{i}_test.tfrecords'
    valid_ds = make_dataset([valid_path], mode='valid')
    train_ds = make_dataset([train_path])
    model = get_model()

    checkpoint = keras.callbacks.ModelCheckpoint(f'model_{i}.tf', monitor='val_correlation', mode='min', 
                                                 save_best_only=True, save_weights_only=True)
    early_stop = keras.callbacks.EarlyStopping(patience=10)
    history = model.fit(train_ds, epochs=30, validation_data=valid_ds, callbacks=[checkpoint, early_stop])
    model.load_weights(f"model_{i}.tf")
    
    for metric in ['loss', 'mae', 'mape', 'rmse', 'correlation']:
        pd.DataFrame(history.history, columns=[metric, f'val_{metric}']).plot()
        plt.title(metric.upper())
        plt.show()

    y_vals = []
    for _, y in valid_ds:
        y_vals += list(y.numpy().reshape(-1))
        
    y_val = np.array(y_vals)
    pearson_score = stats.pearsonr(model.predict(valid_ds).reshape(-1), y_val)[0]
    models.append(model)
    print(f'Pearson Score: {pearson_score}')

Модель показывает не самый лучший, но и не самый плохой результат. Архитектура в целом работает и ее можно улучшить:

<h1>
    <div style="color:black;
           border-radius:5px;
           background-color:#629b50">
4. Future work
</h1>

Эту модель можно улучшить:

1. Увеличить количество слоев
2. Чтобы избежать переобучения увеличить вероятность пропуска нейронов в Dropout
3. Использовать топ-100 признаков из LGBM (обучив на полном датасете)
4. Добавить новые фичи:
* Количество наблюдений в каждый time_id
* Количество наблюдений для каждого актива
* Признаки для временных рядов: несколько предыдущих наблюдений по активу, среднее значение таргета по активу


К сожалению это сделать я не успела, но обязательно попробую. 


<h1>
    <div style="color:black;
           border-radius:5px;
           background-color:#629b50">
Референсы
</h1>


    
[1] https://www.kaggle.com/robikscube/fast-data-loading-and-low-mem-with-parquet-files

[2] https://www.kaggle.com/fangya/ubiquant-investment-mini-eda-lgbm-linreg

[3] https://www.kaggle.com/kartushovdanil/the-most-advanced-analytics

[4] https://www.kaggle.com/lucamassaron/eda-target-analysis

[5] https://www.kaggle.com/lonnieqin/ubiquant-market-prediction-with-dnn

